In [1]:
import pandas as pd
import numpy as np

In [2]:
training_data = np.loadtxt(r"D:\Courses\Projects\Spam emails Classification\training-data.txt", delimiter=' ', dtype=int)
test_data = np.loadtxt(r"D:\Courses\Projects\Spam emails Classification\test-data.txt", delimiter=' ', dtype=int)

In [3]:
training_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [5]:
print('Nr of emails in training file', np.unique(training_data[:, 0]).size)

Nr of emails in training file 4013


In [6]:
print('Nr of emails in test file', np.unique(test_data[:, 0]).size)

Nr of emails in test file 1724


In [7]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, 2500))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [8]:
len(column_names)

2502

In [10]:
index_names = np.unique(training_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [11]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [12]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):

    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, 2500))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [18]:
%%time
full_train_data = make_full_matrix(training_data, 2500)

Wall time: 8.09 s


In [20]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Model

In [21]:
full_train_data.CATEGORY.sum()

1248

In [22]:
full_train_data.CATEGORY.size

4013

In [23]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.310989284824321


In [24]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [26]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [27]:
total_wc = email_lengths.sum()
total_wc

429234

In [28]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]

In [29]:
spam_wc = spam_lengths.sum()
spam_wc

176315

In [30]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [31]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [32]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252919

In [33]:
spam_wc + nonspam_wc - total_wc

0

In [34]:
full_train_features.shape

(4013, 2500)

In [35]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
train_spam_tokens.shape

(1248, 2500)

In [37]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [38]:
summed_spam_tokens.tail()

2495     1
2496     6
2497     1
2498    10
2499     1
dtype: int64

In [39]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [40]:
summed_ham_tokens.tail()

2495    31
2496    21
2497    33
2498    21
2499    33
dtype: int64

In [41]:
train_ham_tokens[2499].sum() + 1

33

## Probability that a token occurs given the email is spam

In [43]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + 2500)
prob_tokens_spam[:5]

0    0.012186
1    0.005229
2    0.006806
3    0.011308
4    0.006817
dtype: float64

In [44]:
prob_tokens_spam.sum()

1.0

## Probability that a token occurs given the email is non-spam

In [46]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + 2500)
prob_tokens_nonspam.sum()

1.0

## Probability that a token occurs

In [47]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0000000000000002

In [48]:
np.savetxt(r"D:\Courses\Projects\Spam emails Classification\prob-spam.txt", prob_tokens_spam)
np.savetxt(r"D:\Courses\Projects\Spam emails Classification\prob-nonspam.txt", prob_tokens_nonspam)
np.savetxt(r"D:\Courses\Projects\Spam emails Classification\prob-all-tokens.txt", prob_tokens_all)

## Preparing test data

In [49]:
test_data.shape

(117777, 4)

In [50]:
%%time
full_test_data = make_full_matrix(test_data, nr_words=2500)

Wall time: 3.89 s


In [51]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [52]:
np.savetxt(r"D:\Courses\Projects\Spam emails Classification\test-target.txt", y_test)
np.savetxt(r"D:\Courses\Projects\Spam emails Classification\test-features.txt", X_test)